In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-12-22 01:32:56--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.60MB/s    in 0.2s    

2022-12-22 01:32:56 (4.60 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

# Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games| 

In [19]:
from pyspark.sql.functions import to_date,col,count
# Read in the Review dataset as a DataFrame

# Deliverable 2

In [6]:
# Create the vine_table DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RSH1OZ87OYK92|          2|            2|          3|   N|                N|
|R1WFOQ3N9BO65I|          5|            0|          0|   N|                Y|
| R3YOOS71KM5M9|          5|            0|          0|   N|                Y|
|R3R14UATT3OUFU|          5|            0|          0|   N|                Y|
| RV2W9SGDNQA2C|          5|            0|          0|   N|                Y|
|R3CFKLIZ0I2KOB|          5|            0|          0|   N|                Y|
|R1LRYU1V0T3O38|          4|            0|          0|   N|                Y|
| R44QKV6FE5CJ2|          5|            0|          0|   N|                Y|
|R2TX1KLPXXXNYS|          5|            0|          0|   N|                Y|
|R1JEEW4C6R89BA|          5|            0|          0|   N|     

In [20]:
# Filter the data and create a new DataFrame
# Retrieve all rows where total_votes count is equal or greater than 20
total_votes_df = vine_df.filter( col("total_votes") >= 20 )
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RV4UH0OWW97K4|          5|           19|         22|   N|                N|
| RG6GT6G9WNYT1|          5|           22|         22|   N|                N|
| RFBXE3Q3GYQYO|          5|           68|         70|   N|                Y|
|R17V95WYC9ZXDD|          1|            0|         20|   N|                Y|
|R235HEIYY15NZX|          1|            0|         26|   N|                Y|
|R3BZ4DHX2YHC4V|          1|            1|         32|   N|                Y|
| RZSC8Q4630BV1|          1|            1|         26|   N|                N|
|R34S64NPUZOS90|          5|           23|         24|   N|                N|
|R35O3GV3HZUX8B|          5|           58|         63|   N|                N|
|R2AZAMZCEUOKQT|          1|           29|         40|   N|     

In [11]:
# Using total_votes_df, create a new DF with the rows where the helpful_votes / total_votes >= 50% 
helpful_votes_df = total_votes_df.withColumn('helpful_votes_percent', col("helpful_votes")/col("total_votes") )
helpful_votes_df = helpful_votes_df.filter(col("helpful_votes_percent") >= 0.5)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_votes_percent|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
| RV4UH0OWW97K4|          5|           19|         22|   N|                N|   0.8636363636363636|
| RG6GT6G9WNYT1|          5|           22|         22|   N|                N|                  1.0|
| RFBXE3Q3GYQYO|          5|           68|         70|   N|                Y|   0.9714285714285714|
|R34S64NPUZOS90|          5|           23|         24|   N|                N|   0.9583333333333334|
|R35O3GV3HZUX8B|          5|           58|         63|   N|                N|   0.9206349206349206|
|R2AZAMZCEUOKQT|          1|           29|         40|   N|                Y|                0.725|
|R103RUVEH5YWQ4|          1|           69|         90|   N|                Y|   0.7666666666666667|


In [14]:
# Filter helpful_votes_df and create a new DataFrame with all rows that are a Vine
vine_true_review_df = helpful_votes_df.filter(col("vine") == "Y")
vine_true_review_df.show()

+---------+-----------+-------------+-----------+----+-----------------+---------------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_votes_percent|
+---------+-----------+-------------+-----------+----+-----------------+---------------------+
+---------+-----------+-------------+-----------+----+-----------------+---------------------+



In [16]:
# Filter helpful_votes_df and create a new DataFrame with all rows that are NOT a Vine
vine_false_review_df = helpful_votes_df.filter(col("vine") == "N")
vine_false_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_votes_percent|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
| RV4UH0OWW97K4|          5|           19|         22|   N|                N|   0.8636363636363636|
| RG6GT6G9WNYT1|          5|           22|         22|   N|                N|                  1.0|
| RFBXE3Q3GYQYO|          5|           68|         70|   N|                Y|   0.9714285714285714|
|R34S64NPUZOS90|          5|           23|         24|   N|                N|   0.9583333333333334|
|R35O3GV3HZUX8B|          5|           58|         63|   N|                N|   0.9206349206349206|
|R2AZAMZCEUOKQT|          1|           29|         40|   N|                Y|                0.725|
|R103RUVEH5YWQ4|          1|           69|         90|   N|                Y|   0.7666666666666667|


In [23]:
# Determine the total number of reviews, the number of 5-star reviews,
# and the percentage of 5-star reviews for the two types of reviews

# Total review values for vine true and false
vtrue_total_reviews = vine_true_review_df.count()
vfalse_total_reviews = vine_false_review_df.count()

# Total five star reviews for vine true and false
vtrue_fivestar_reviews = vine_true_review_df.filter("star_rating == 5").count()
vfalse_fivestar_reviews = vine_false_review_df.filter("star_rating == 5").count()

# Percentage of five star reviews for vine true and flase
try:
  vtrue_percent_fivestar = vtrue_fivestar_reviews / vtrue_total_reviews
except ZeroDivisionError:
  vtrue_percent_fivestar = 0

try:
  vfalse_percent_fivestar = vfalse_fivestar_reviews / vfalse_total_reviews
except ZeroDivisionError:
  vfalse_percent_fivestar = 0

In [31]:
import pandas as pd

# Setup data for DataFrame
data = {
    'vine': ['Y', 'N'],
    'total_reviews': [vtrue_total_reviews, vfalse_total_reviews],
    'total_fivestar': [vtrue_fivestar_reviews, vfalse_fivestar_reviews],
    'percentage_fivestar': [vtrue_percent_fivestar, vfalse_percent_fivestar]
}

# Create DataFrame
result_df = pd.DataFrame.from_dict(data)

# Show DataFrame
result_df

,vine,total_reviews,total_fivestar,percentage_fivestar
0,Y,0,0,0.000000
1,N,1685,631,0.374481
